In [21]:
from world_model import WorldModel2
import tensorflow_datasets as tfds
from rlu_unplugged_transform import batch_dataset

In [22]:
data = batch_dataset(tfds.load("rlu_atari", split="train[:5%]"))
k = WorldModel2()
k.compile(optimizer='adam', loss='mse')

2023-03-22 19:14:46.224729: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [23]:
k.fit(data, epochs=1)

: 

: 